# Metadata vector mask and data raster mask

[xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) can handle several masks. 

A mask is a shapely polygon within the [xsar.SentinelMeta.footprint](../basic_api.rst#xsar.SentinelMeta.footprint).
Default mask is 'land', but the user can add customs masks by providing a [cartopy.feature.Feature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) or a shapefile to [xsar.SentinelMeta.set_mask_feature](../basic_api.rst#xsar.SentinelMeta.set_mask_feature).

A mask can be retrieved with [xsar.SentinelMeta.get_mask](../basic_api.rst#xsar.SentinelMeta.get_mask).

When building an xarray dataset from a [xsar.SentinelMeta](../basic_api.rst#xsar.SentinelMeta) object with  [xsar.open_dataset](../basic_api.rst#xsar.open_dataset), masks are rasterized, to add a variable like 'land_mask' to the final dataset. (Not yet implemented)



In [ ]:
import geopandas as gpd
import pandas as pd
import datetime
import xsar
from osgeo import ogr, gdal
import shapely
import cartopy
import os
import datetime
import rasterio
import rasterio.features

In [ ]:
# use holoviews for plots
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
hv.extension('bokeh')
gv.extension('bokeh')
from holoviews.operation.datashader import datashade,rasterize

## Metadata vector mask

[xsar.SentinelMeta.footprint](../basic_api.rst#xsar.SentinelMeta.footprint) is the aquisition footprint, e.g. a rectangle around the acquisition. 

Metadata masks are a [shapely.geometry](https://shapely.readthedocs.io/) object, computed from the intersection between [xsar.SentinelMeta.footprint](../basic_api.rst#xsar.SentinelMeta.footprint) and a [cartopy.feature.Feature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) object. 

In [ ]:
# here is the acquisition footprint, whithout land information
filename = xsar.get_test_file('S1B_IW_GRDH_1SDV_20181013T062322_20181013T062347_013130_018428_Z000.SAFE')
s1meta = xsar.SentinelMeta(filename)
s1meta.footprint

A mask is a shape within [xsar.SentinelMeta.footprint](../basic_api.rst#xsar.SentinelMeta.footprint) that can be retrieved by [xsar.SentinelMeta.get_mask](../basic_api.rst#xsar.SentinelMeta.get_mask)

In [ ]:
default_land_footprint = s1meta.get_mask('land')
default_land_footprint

A mask can be changed or added with [xsar.SentinelMeta.set_mask_feature](../basic_api.rst#xsar.SentinelMeta.set_mask_feature). 

Here, we are defining a new 'ocean' mask: 

In [ ]:
s1meta.set_mask_feature('ocean', cartopy.feature.OCEAN)
s1meta.get_mask('ocean')

### High resolution mask

Default [cartopy.feature.LAND](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#cartopy.feature.LAND) is low resolution, but [xsar.SentinelMeta.set_mask_feature](../basic_api.rst#xsar.SentinelMeta.set_mask_feature) allow to give better resolution features with [cartopy.feature.Feature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html) or a user defined shapefile.



  - NaturalEarthFeature

With [cartopy.feature.NaturalEarthFeature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#cartopy.feature.NaturalEarthFeature)

In [ ]:
# the 'land' mask is ovewritten
s1meta.set_mask_feature('land', cartopy.feature.NaturalEarthFeature('physical', 'land', '10m'))
ne_10m_land_footprint = s1meta.get_mask('land')
ne_10m_land_footprint

  - GSHHSFeature

With [cartopy.feature.GSHHSFeature](https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html#cartopy.feature.GSHHSFeature)

In [ ]:
s1meta.set_mask_feature('land',cartopy.feature.GSHHSFeature(scale='full'))
gshhs_land_footprint = s1meta.get_mask('land')
gshhs_land_footprint

  - Custom feature from shapefile (openstreetmap)

Cartopy has no backends for [openstreetmap shapefiles](https://wiki.openstreetmap.org/wiki/Shapefiles), but we can give any shapefile that can be read by [geopandas.read_file](https://geopandas.org/reference/geopandas.read_file.html#geopandas-read-file)  to 
 [xsar.SentinelMeta.set_mask_feature](../basic_api.rst#xsar.SentinelMeta.set_mask_feature)  

>**Optimisation note**
>
> *  Only shapes intersecting with footprint will be read in the shapefile. So performances will be better if shapes are split in smaller shapes (e.g. Eurasian polygon is not an huge polygon)
> *  A [splitted version](https://osmdata.openstreetmap.de/data/land-polygons.html) from openstreetmap shapefile has been used to speedup access.


In [ ]:
s1meta.set_mask_feature('land', os.path.join(xsar.get_test_file('land-polygons-split-4326'),'land_polygons.shp'))
osm_land_footprint = s1meta.get_mask('land')
osm_land_footprint


#### Resolutions comparison

(Use the mouse to change zoom level)

In [ ]:
(gv.Shape(default_land_footprint).opts(title='default') \
+ gv.Shape(ne_10m_land_footprint).opts(title='ne_10m') \
+ gv.Shape(gshhs_land_footprint).opts(title='gshhs_full') \
+ gv.Shape(osm_land_footprint).opts(title='osm')).cols(2)

### Coordinates Reference System

A metadata mask is defined in longitude/latitude reference (EPSG:4326), but the sar data has no CRS.
However, sar data has Ground Control Points (GCP), that can be used to convert longitude and latitude to and from atrack and xtrack, with resp [s1meta.ll2coords](../basic_api.rst#xsar.SentinelMeta.ll2coords) and [s1meta.ll2coords](../basic_api.rst#xsar.SentinelMeta.coords2ll)

In [ ]:
osm_land_footprint_ax = s1meta.ll2coords(osm_land_footprint)
osm_land_footprint_ax

So we are now able to plot the mask over the data

In [ ]:
ds = xsar.open_dataset(s1meta)
rasterize(hv.Image(((ds.digital_number.sel(pol='VH'))**(1/5))).persist().opts(cmap='gray',colorbar=True,tools=['hover'],title="xsar")) \
* hv.Path(osm_land_footprint_ax).opts(color='lightgreen',width=800,height=800) 

## Data raster mask

Data raster mask is currently not implemented.
However, here is a method to get it. ( **Warning** : this is in-memory, non optimized for dask ! )

In [ ]:
raster = rasterio.features.rasterize([osm_land_footprint_ax],out_shape=(ds.incidence.shape[1],ds.incidence.shape[0])).T
ds['land_mask'] = ( {'atrack': raster.shape[1] , 'xtrack': raster.shape[0]}, raster)
ds['dn_masked'] = (ds.digital_number.sel(pol='VH')**(1/5)).where(ds['land_mask'] == 0).persist()
rasterize(hv.Image(ds['dn_masked'])).opts(cmap='gray', width=800,height=800) 

## Perfomance comparison

In [ ]:
# Following code is to compare performances for different methods: it's not usefull for the end user

index = ['default', 'ne_10m_land', 'gshhs_full_land' , 'osm']
columns = ['feature_f', 'feature_init_time', 'land_footprint_time', 'land_footprint_ax_time', 'rasterize_time']

land_features_df = pd.DataFrame(index=index, columns=columns)
land_features_df.loc['default', 'feature_f'] = lambda: cartopy.feature.LAND
land_features_df.loc['ne_10m_land', 'feature_f'] = lambda: cartopy.feature.NaturalEarthFeature('physical', 'land', '10m')
land_features_df.loc['gshhs_full_land', 'feature_f'] = lambda: cartopy.feature.GSHHSFeature(scale='full')
land_features_df.loc['osm', 'feature_f'] = lambda: os.path.join(xsar.get_test_file('land-polygons-split-4326'),'land_polygons.shp')

raster_shape = s1meta.rio.shape
for feature_idx, feature_rows in land_features_df.iterrows():
    t1 = datetime.datetime.now()
    s1meta.set_mask_feature('land',feature_rows['feature_f']())
    land_features_df.at[feature_idx, 'feature_init_time'] = datetime.datetime.now() - t1
    t1 = datetime.datetime.now()
    land_footprint = s1meta.get_mask('land')
    land_features_df.at[feature_idx, 'land_footprint_time'] = datetime.datetime.now() - t1
    t1 = datetime.datetime.now()
    land_footprint_ax = s1meta.ll2coords(land_footprint)
    land_features_df.at[feature_idx, 'land_footprint_ax_time'] = datetime.datetime.now() - t1
    t1 = datetime.datetime.now()
    raster = rasterio.features.rasterize([land_footprint],out_shape=raster_shape)
    land_features_df.at[feature_idx, 'rasterize_time'] = datetime.datetime.now() - t1
land_features_df
